In [ ]:
#Function from stackOverflow to calculate quaternian
def quaternion_to_euler_angle_vectorized1(w, x, y, z):
    ysqr = y * y

    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + ysqr)
    X = np.degrees(np.arctan2(t0, t1))

    t2 = +2.0 * (w * y - z * x)
    t2 = np.where(t2>+1.0,+1.0,t2)
    #t2 = +1.0 if t2 > +1.0 else t2

    t2 = np.where(t2<-1.0, -1.0, t2)
    #t2 = -1.0 if t2 < -1.0 else t2
    Y = np.degrees(np.arcsin(t2))

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (ysqr + z * z)
    Z = np.degrees(np.arctan2(t3, t4))

    return X, Y, Z 

import csv

#fileList = ('SS1', 'SS2','SS3','SS4')
fileList = ('SS3','SS3_ke')
timeList = []
heaveList = []
pitchList = []

for i in range(len(fileList)):
    q0=[]
    q1=[]
    q2=[]
    q3=[]
    
    tempTimeList = []
    tempPitchList = []
    tempHeaveList = []
    eulers=[]
    with open(fileList[i]+".csv", 'r') as f:
        d_reader = csv.DictReader(f)
    
        #get fieldnames from DictReader object and store in list
        headers = d_reader.fieldnames
        counter = 0
        for line in d_reader:
            if counter ==0:
                initialHeave = float(line['z']) 
                counter+=1
            #print value in MyCol1 for each row
            q0.append(float(line['rotq_e0']))
            q1.append(float(line['rotq_e1']))
            q2.append(float(line['rotq_e2']))
            q3.append(float(line['rotq_e3']))
            tempTimeList.append(float(line['t']))
            tempHeaveList.append(float(line['z'])-initialHeave)
    timeList.append(tempTimeList)
    heaveList.append(tempHeaveList)
    import numpy as np
    
    
    for i in range(len(q0)):
        eulers.append(quaternion_to_euler_angle_vectorized1(q0[i],q1[i],q2[i],q3[i]))
        
        

    for i in range(len(eulers)):
        tempPitchList.append(eulers[i][1]*180/np.pi)
    pitchList.append(tempPitchList)
    

import matplotlib.pyplot as plt

dpi = 150
import matplotlib as mpl
mpl.rcParams['figure.dpi']= dpi

for i in range(len(fileList)):
    plt.plot(timeList[i],pitchList[i], label=fileList[i])
plt.ylabel('Pitch, degrees')
plt.xlabel('Time, seconds')
plt.title('Pitch vs. Time')
plt.xlim([0, 40])
plt.legend()
plt.show()


maxPitch = []
for i in range(len(fileList)):
    tempMaxPitch = 0
    for j in range(len(pitchList[i])):
        if abs(pitchList[i][j]) > tempMaxPitch:
            tempMaxPitch = abs(pitchList[i][j])
    maxPitch.append(tempMaxPitch)

for i in range(len(fileList)):
    print("The max pitch for "+fileList[i]+ " is " +str(round(maxPitch[i],2)) + " degrees.")

    
for i in range(len(fileList)):
    plt.plot(timeList[i],heaveList[i], label=fileList[i])
plt.ylabel('Heave, meters')
plt.xlabel('Time, seconds')
plt.title('Heave vs. Time')
plt.xlim([0, 40])
plt.legend()
plt.show()

maxHeave = []
for i in range(len(fileList)):
    tempMaxHeave = 0
    for j in range(len(heaveList[i])):
        if abs(heaveList[i][j]) > tempMaxHeave:
            tempMaxHeave = abs(heaveList[i][j])
    maxHeave.append(tempMaxHeave)

for i in range(len(fileList)):
    print("The max heave for "+fileList[i]+ " is " +str(round(maxHeave[i],4)) + " meters.")

In [ ]:
maxPitch